In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import string
import tweedie
#from pandas_prof
import importlib
import os, sys
from pathlib import Path
#sys.path.append(Path(os.getcwd()).parents[1])
import dsutils

In [2]:
from dsutils.dsutils.transformers import *
#from dsutils.dsutils.MissingHandler import MissingHandler
from dsutils.dsutils.helpers import histograms
from dsutils.dsutils.pipeline import Pipeline

In [3]:
BaseTransformer?

Init signature: BaseTransformer()
Docstring:      Base class for all transformers
File:           c:\users\grego\documents\github\dsutils\dsutils\transformers\_base.py
Type:           type
Subclasses:     _CategoricalBinner, BaseMissingTransformer


In [4]:
n=100
np.random.seed(12345)
df = pd.DataFrame(
{
    'w': np.random.choice(range(10),n).astype(float),
    'x': np.random.choice(list(string.ascii_lowercase),n),
    'y': np.random.normal(size=n),
    'z': tweedie.tweedie(mu=10, p=1.5, phi=20).rvs(size = n)
}
)

na_mask = (np.random.binomial(n=1,p=0.2,size = df.shape)).astype(bool)
df[na_mask] = np.nan

df

C:\Users\grego\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:998: VisibleDeprecationWarning: The signature of <bound method tweedie_gen._rvs of <tweedie.tweedie_dist.tweedie_gen object at 0x000001DA525D4B08>> does not contain a "size" keyword.  Such signatures are deprecated.
  np.VisibleDeprecationWarning)


,w,x,y,z
0,2.0,w,NaN,NaN
1,NaN,f,-0.402827,0.000000
2,1.0,t,NaN,0.000000
3,NaN,NaN,-0.193401,NaN
4,9.0,c,0.669158,5.421594
...,...,...,...,...
95,NaN,NaN,-0.132609,0.000000
96,2.0,c,1.457300,7.690127
97,0.0,i,0.609512,0.000000
98,NaN,q,-0.493779,0.000000


In [8]:
p = Pipeline([
    ("fill_cat_nas",ReplaceMissingCategorical(),'x'),
    ("pcb",PercentThresholdBinner(percent_threshold = 0.03),'x'),
    ("rmmean",ReplaceMissingMean(),['w','y','z'])
])

In [9]:
p.fit(df)

In [10]:
p.transform(df)

,w,x,y,z
0,2.000000,w,-0.005251,6.387428
1,4.178082,f,-0.402827,0.000000
2,1.000000,t,-0.005251,0.000000
3,4.178082,_MISSING_,-0.193401,6.387428
4,9.000000,c,0.669158,5.421594
...,...,...,...,...
95,4.178082,_MISSING_,-0.132609,0.000000
96,2.000000,c,1.457300,7.690127
97,0.000000,_OTHER_,0.609512,0.000000
98,4.178082,q,-0.493779,0.000000


In [17]:
p._steps[1][1]._fitted

False

In [16]:
p._steps[0][1].transform(df)

,w,x,y,z
0,2.0,w,NaN,NaN
1,NaN,f,-0.402827,0.000000
2,1.0,t,NaN,0.000000
3,NaN,NaN,-0.193401,NaN
4,9.0,c,0.669158,5.421594
...,...,...,...,...
95,NaN,NaN,-0.132609,0.000000
96,2.0,c,1.457300,7.690127
97,0.0,_OTHER_,0.609512,0.000000
98,NaN,q,-0.493779,0.000000


In [8]:
pcb = PercentThresholdBinner(percent_threshold = 0.03)

In [9]:
pcb.fit(df,'x')

In [10]:
df2 = pcb.transform(df)

In [11]:
df2

,w,x,y,z
0,2.0,w,NaN,NaN
1,NaN,f,-0.402827,0.000000
2,1.0,t,NaN,0.000000
3,NaN,NaN,-0.193401,NaN
4,9.0,c,0.669158,5.421594
...,...,...,...,...
95,NaN,NaN,-0.132609,0.000000
96,2.0,c,1.457300,7.690127
97,0.0,_OTHER_,0.609512,0.000000
98,NaN,q,-0.493779,0.000000


In [17]:
df2[df2.x.isna()]

,w,x,y,z
3,NaN,NaN,-0.193401,NaN
8,6.0,NaN,0.353607,0.000000
14,2.0,NaN,-0.417064,18.017893
20,7.0,NaN,0.457940,0.000000
21,7.0,NaN,NaN,NaN
22,8.0,NaN,1.306720,0.000000
28,NaN,NaN,NaN,0.000000
32,1.0,NaN,NaN,0.000000
34,2.0,NaN,0.680321,126.477892
39,2.0,NaN,-1.677790,0.000000


In [8]:
df2 = df.copy()
grouper = Grouper()
mh = MissingHandler()
mh.fit_missing_ind(df2,x=['w','y','z'])
mh.fit_missing_string(df2,x='x')
mh.fit_missing_numeric(df2,x=['w','y','z'])

In [9]:
df2

,w,x,y,z,w_NA_IND,y_NA_IND,z_NA_IND
0,2.0,w,-1.000000,-1.000000,0,1,1
1,-1.0,f,-0.402827,0.000000,1,0,0
2,1.0,t,-1.000000,0.000000,0,1,0
3,-1.0,_MISSING_,-0.193401,-1.000000,1,0,1
4,9.0,c,0.669158,5.421594,0,0,0
...,...,...,...,...,...,...,...
95,-1.0,_MISSING_,-0.132609,0.000000,1,0,0
96,2.0,c,1.457300,7.690127,0,0,0
97,0.0,i,0.609512,0.000000,0,0,0
98,-1.0,q,-0.493779,0.000000,1,0,0


In [10]:
grouper.fit(df2,x='x',kind='max_levels')

In [11]:
df2

,w,x,y,z,w_NA_IND,y_NA_IND,z_NA_IND
0,2.0,w,-1.000000,-1.000000,0,1,1
1,-1.0,f,-0.402827,0.000000,1,0,0
2,1.0,t,-1.000000,0.000000,0,1,0
3,-1.0,_MISSING_,-0.193401,-1.000000,1,0,1
4,9.0,c,0.669158,5.421594,0,0,0
...,...,...,...,...,...,...,...
95,-1.0,_MISSING_,-0.132609,0.000000,1,0,0
96,2.0,c,1.457300,7.690127,0,0,0
97,0.0,i,0.609512,0.000000,0,0,0
98,-1.0,q,-0.493779,0.000000,1,0,0


In [13]:
#grouper.info.pop('y',None)

grouper.info

{'x': {'grouper': {'kind': 'max_levels',
   'max_levels': 10,
   'levels': ['_MISSING_', 'z', 'f', 'a', 'm', 'w', 't', 'r', 'n', 'c'],
   'other_val': '_OTHER_'}}}

In [14]:
grouper.transform_all(df2)
df2

,w,x,y,z,w_NA_IND,y_NA_IND,z_NA_IND
0,2.0,w,-1.000000,-1.000000,0,1,1
1,-1.0,f,-0.402827,0.000000,1,0,0
2,1.0,t,-1.000000,0.000000,0,1,0
3,-1.0,_MISSING_,-0.193401,-1.000000,1,0,1
4,9.0,c,0.669158,5.421594,0,0,0
...,...,...,...,...,...,...,...
95,-1.0,_MISSING_,-0.132609,0.000000,1,0,0
96,2.0,c,1.457300,7.690127,0,0,0
97,0.0,_OTHER_,0.609512,0.000000,0,0,0
98,-1.0,_OTHER_,-0.493779,0.000000,1,0,0


In [15]:
mh.info

{'w': {'missing_ind': {'postfix': '_NA_IND'},
  'fit_missing_numeric': {'fill_val': -1}},
 'y': {'missing_ind': {'postfix': '_NA_IND'},
  'fit_missing_numeric': {'fill_val': -1}},
 'z': {'missing_ind': {'postfix': '_NA_IND'},
  'fit_missing_numeric': {'fill_val': -1}},
 'x': {'fit_missing_string': {'fill_val': '_MISSING_',
   'missing_vals': ['', 'UNK', 'UNKNOWN'],
   'case_sensitive': False}}}

,w,x,y,z
0,2,w,NaN,NaN
1,<NA>,f,-0.402827,0.000000
2,1,t,NaN,0.000000
3,<NA>,NaN,-0.193401,NaN
4,9,c,0.669158,5.421594
...,...,...,...,...
95,<NA>,NaN,-0.132609,0.000000
96,2,c,1.457300,7.690127
97,0,i,0.609512,0.000000
98,<NA>,q,-0.493779,0.000000


In [84]:
(-df.loc[:,'w'].isna()).astype(int)

0     1
1     0
2     1
3     0
4     1
     ..
95    0
96    1
97    1
98    0
99    1
Name: w, Length: 100, dtype: int32

In [36]:
def col_types(df):
    {col:pd.api.types.is_numeric_dtype(df.loc[:,col])
     for col in df.columns}


for col in df.columns:
    print(col)
    print(pd.api.types.is_numeric_dtype(df.loc[:,col]))

x
False
y
True
z
True


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       78 non-null     object 
 1   y       82 non-null     float64
 2   z       81 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [95]:
d = df.dtypes.to_dict()

In [99]:
pd.api.types.is_numeric_dtype(d['w'])

True

In [38]:
df.describe()

,y,z
count,82.000000,81.000000
mean,0.115844,7.163064
std,1.047992,18.269160
min,-2.979359,0.000000
25%,-0.455898,0.000000
50%,0.048757,0.000000
75%,0.837086,0.867553
max,2.613154,92.670684


In [149]:
z = 'x'
-df[z].isna()
df[z].transform(lambda l: str(l).upper() in ['W']) | df[z].isna()

0      True
1     False
2     False
3      True
4     False
      ...  
95     True
96    False
97    False
98    False
99    False
Name: x, Length: 100, dtype: bool

In [127]:
k = {'x':1,'y':2}
list(k.keys())

['x', 'y']

In [104]:
missing_vals = ["","UNK","UNKNOWN"]
fill_val = "_MISSING_"
pd.DataFrame({'x':['','Unknown','z',np.nan]})['x'] \
.fillna(fill_val) \
.transform(lambda z: fill_val if z.upper() in missing_vals else z)

0    _MISSING_
1    _MISSING_
2            z
3    _MISSING_
Name: x, dtype: object